# Introduction to the Regime-Aware Pricing System

Welcome! This notebook will walk you through the basics of the system.

## What You'll Learn
1. What the system does (high-level)
2. How to load and visualize data
3. Basic concepts (log returns, jumps, volatility)
4. Running a simple example

## Prerequisites
- Basic Python knowledge
- Understanding of stock prices
- Curiosity!

Let's get started! 🚀

## 1. Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Add modules directory to path
sys.path.append('../modules')

# Import our custom modules
from jump_detector import JumpDetector, calculate_log_returns
from volatility_engine import VolatilityEngine

# Set up plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Setup complete!")

## 2. Generate Synthetic Data

For learning purposes, let's create fake stock price data.

This simulates:
- Normal daily movements (diffusion)
- Occasional large jumps (crashes/spikes)

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Parameters
n_days = 500
initial_price = 100
daily_drift = 0.0005  # 0.05% average daily return
daily_vol = 0.015     # 1.5% daily volatility

# Generate normal returns
normal_returns = np.random.normal(daily_drift, daily_vol, n_days)

# Add 10 random jumps (big moves)
jump_days = np.random.choice(n_days, size=10, replace=False)
for day in jump_days:
    jump_size = np.random.choice([-1, 1]) * np.random.uniform(0.04, 0.10)
    normal_returns[day] += jump_size

# Convert returns to prices
prices = initial_price * np.exp(np.cumsum(normal_returns))

# Create DataFrame
dates = pd.date_range('2022-01-01', periods=n_days, freq='D')
df = pd.DataFrame({
    'Price': prices
}, index=dates)

print(f"Generated {n_days} days of price data")
print(f"Starting price: ${initial_price:.2f}")
print(f"Ending price: ${prices[-1]:.2f}")
print(f"Total return: {100 * (prices[-1] / initial_price - 1):.2f}%")

## 3. Visualize the Price Series

Let's see what our fake stock looks like.

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Price'], linewidth=1.5)
plt.title('Simulated Stock Price', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Can you spot the jumps? They're the big sudden moves!")

## 4. Calculate Log Returns

**What are log returns?**
- Instead of looking at raw prices, we look at % changes
- Formula: `r = log(Price_today / Price_yesterday)`
- Makes the math easier and more stable

**Why log returns?**
- Symmetric (10% up then 10% down = back to start)
- Time-additive (can sum them)
- Better statistical properties

In [ ]:
# Calculate returns
returns = calculate_log_returns(df['Price'])

print("Return Statistics:")
print(f"  Mean: {returns.mean():.6f} (daily average)")
print(f"  Std:  {returns.std():.6f} (daily volatility)")
print(f"  Min:  {returns.min():.6f} (worst day)")
print(f"  Max:  {returns.max():.6f} (best day)")

# Plot returns
plt.figure(figsize=(14, 6))
plt.plot(returns.index, returns.values, alpha=0.7, linewidth=0.8)
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.title('Daily Returns', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Log Return')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Detect Jumps

**What are jumps?**
- Unusually large price movements
- Often caused by news, crises, or major events
- Need to separate them from normal volatility

**How do we detect them?**
- Statistical threshold: if |return| > 3 standard deviations → jump
- Uses rolling window to adapt to changing conditions

In [ ]:
# Initialize jump detector
detector = JumpDetector(
    threshold=3.0,  # 3 standard deviations
    window=20       # 20-day rolling window
)

# Detect jumps
is_jump, jump_returns, diffusion_returns = detector.detect_jumps(returns)

# Print results
print(f"Jumps detected: {is_jump.sum()} out of {len(returns)} days")
print(f"Jump percentage: {100 * is_jump.sum() / len(returns):.2f}%")

# Get jump parameters
jump_params = detector.estimate_jump_parameters(jump_returns)
print(f"\nJump Statistics:")
print(f"  Mean size: {jump_params['mu_j']:.4f}")
print(f"  Volatility: {jump_params['sigma_j']:.4f}")
print(f"  Frequency: {jump_params['lambda']:.4f} jumps/day")

In [ ]:
# Visualize jumps
plt.figure(figsize=(14, 6))
plt.plot(returns.index, returns.values, alpha=0.5, label='All Returns')
plt.scatter(
    returns.index[is_jump],
    returns.values[is_jump],
    color='red',
    s=100,
    label='Detected Jumps',
    zorder=5
)
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.title('Jump Detection Results', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Log Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Red points = detected jumps!")

## 6. Estimate Volatility

**What is volatility?**
- Measure of how much prices swing around
- High volatility = risky, unpredictable
- Low volatility = stable, predictable

**Why separate jumps?**
- Jumps contaminate volatility estimates
- We only use diffusion returns (jumps removed)
- This gives cleaner risk measurements

In [ ]:
# Initialize volatility engine
vol_engine = VolatilityEngine(
    bull_ewma_lambda=0.94  # Slow decay for stable periods
)

# Estimate volatility (using diffusion returns only!)
volatility = vol_engine.ewma_volatility(
    diffusion_returns,  # Jumps removed!
    lambda_decay=0.94
)

# Statistics
print("Volatility Statistics:")
print(f"  Mean: {volatility.mean():.4f}")
print(f"  Current: {volatility.iloc[-1]:.4f}")
print(f"  Annualized (current): {volatility.iloc[-1] * np.sqrt(252):.2%}")

In [ ]:
# Visualize volatility
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Returns
axes[0].plot(returns.index, returns.values, alpha=0.5)
axes[0].scatter(
    returns.index[is_jump],
    returns.values[is_jump],
    color='red',
    s=50,
    label='Jumps'
)
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[0].set_ylabel('Return')
axes[0].set_title('Returns with Jump Detection')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Volatility
axes[1].plot(volatility.index, volatility.values, color='purple', linewidth=2)
axes[1].set_ylabel('Volatility (σ)')
axes[1].set_xlabel('Date')
axes[1].set_title('Estimated Volatility (Jumps Excluded)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Notice: Volatility is smoother because jumps are excluded!")

## 7. Key Takeaways

You've just learned the first three steps of the system:

1. **Log Returns**: Transform prices into % changes
2. **Jump Detection**: Separate big moves from normal volatility
3. **Volatility Estimation**: Measure risk using clean data

### What's Next?

The remaining components:
- **Drift Estimation**: Track directional bias (Kalman filter)
- **Regime Detection**: Identify Bull/Sideways/Crisis markets
- **Option Pricing**: Black-Scholes + Merton jump-diffusion
- **Mispricing**: Compare model vs market prices
- **Kelly Sizing**: Smart position sizing with risk control
- **Decision Engine**: Final BUY/SELL/REFUSE output

### Practice Exercises

1. Try changing the `threshold` parameter in jump detection (2.0, 3.0, 4.0)
2. Experiment with different volatility decay rates (0.90, 0.94, 0.98)
3. Generate your own synthetic data with different parameters
4. Load real stock data and run the same analysis

### Next Notebook

Continue to `02_regime_detection.ipynb` to learn about market regimes!

In [ ]:
print("\n🎉 Congratulations! You've completed the introduction.")
print("\n📚 Keep exploring and learning. Take your time!")